<a href="https://colab.research.google.com/github/camrobertson/FDA-PMR-RWE-Analysis-2018_06_16-/blob/master/FDA_sup_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ML analysis to predict timing/nature of supplements following BLA/NDA submission

Input from drugs@fda.com
Highlight variables BLA/NDA, Date of Submission,, Therapeutic Area (review division), priority review, breakthrough therapy, accelerated approval
See what happens (start w/ RFS)

In [2]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
!pip install sklearn
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from google.colab import drive
from oauth2client.client import GoogleCredentials

import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib as mpl #only needed to determine Matplotlib version number
import csv

     |████████████████████████████████| 993kB 2.7MB/s 


In [0]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
# Might be redundant w/ Cell #5
#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
#works to import data
Products_df = pd.read_csv('/content/gdrive/My Drive/FDA_data/target/Products.txt', '\t', low_memory = False, error_bad_lines = False, encoding = "utf8")
Products_df.describe()
#Products_df

#works to import data
Submissions_df = pd.read_csv('/content/gdrive/My Drive/FDA_data/target/Submissions.txt', '\t', low_memory = False, error_bad_lines = False, encoding='ISO-8859-1')
#Submissions_df.describe()

#works to import data
Applications_df = pd.read_csv('/content/gdrive/My Drive/FDA_data/target/Applications.txt', '\t', low_memory = False, error_bad_lines = False, encoding='ISO-8859-1')
#Applications_df.describe()

Submission Type Binary: 
*   Original Submission = 0
*   Supplement = 1

Application Type
* NDA = 0
* BLA = 1

ReviewPriority
* Standard = 0
* Unknown = 0
* Priority = 1

NaN = 0 Across all



In [0]:
Master_df = Submissions_df.set_index('ApplNo').join(Applications_df.set_index('ApplNo'))
Master_df = Master_df[Master_df.ApplType != "ANDA"]

mask = (Master_df["ApplType"] == "ANDA") #remove ANDA applications
print (Master_df.loc[mask])

In [0]:
Master_df['SubmissionType'] = Master_df['SubmissionType'].map({'ORIG': 0, 'SUPPL': 1})
Master_df['ApplType'] = Master_df['ApplType'].map({'NDA': 0, 'BLA': 1})
Master_df['ReviewPriority'] = Master_df['ReviewPriority'].map({'STANDARD':0, 'UNKNOWN' :0, 'PRIORITY' :1})

Master_df.fillna(0, inplace = True)

Master_df['ApplType'] = Master_df['ApplType'].astype(int)
Master_df['SubmissionClassCodeID'] = Master_df['SubmissionClassCodeID'].astype(int)
Master_df['ReviewPriority'] = Master_df['ReviewPriority'].astype(int)
#Master_df['SubmissionStatus'] = pd.to_datetime(Master_df['SubmissionStatus'], errors="ignore")

In [0]:
#reset index of Master Df
import numpy as np
Master_df['Index'] = np.arange(len(Master_df))
Master_df['ApplNo'] = Master_df.index
Master_df = Master_df.set_index('Index')
Master_df['SubmissionStatusDate'] = pd.to_datetime(Master_df['SubmissionStatusDate']) 
Master_df

Select all submissions after PDUFA V ... can rewind this further back if needed.

In [0]:
start_date = '2007-01-01'
mask = (Master_df['SubmissionStatusDate'] > start_date)
Master_df = Master_df.loc[mask]

Just a quick peek to see how the submissions played out over time

In [0]:
import matplotlib.pyplot as plt
plt.scatter(Master_df['SubmissionStatusDate'], Master_df['SubmissionClassCodeID'])
plt.show()


Next tasks

1.   Determine difference between approval date and submission date
2.   Create / select Data sets by submission class code
3.   Run model RFS test, see how it goes... ?


In [0]:
Original_df = Master_df.loc[Master_df['SubmissionType'] == 0, ['ApplNo', 'SubmissionStatusDate']]
Original_df = Original_df.rename(columns={"SubmissionStatusDate": "OriginalSubDate"})
Original_df

In [0]:
Master_df = Original_df.merge(Master_df, on="ApplNo", how='right')
Master_df['Elapsed_days']= (Master_df['SubmissionStatusDate'] - Master_df['OriginalSubDate']).dt.days
Master_df

To do - need to reformat the elapsed days into a list associated with each original application. 

In [0]:
#Master_df.SubmissionClassCodeID.describe()
pd.Series(Master_df['SubmissionClassCodeID']).value_counts()

Create a subset of DF to test against ML
Need to bin the elapsed days

In [0]:
Testing_df = Master_df[['ApplNo', 'Elapsed_days', 'SubmissionClassCodeID', 'SubmissionType', 'SubmissionNo', 'ReviewPriority', 'ApplType']]
Testing_df.head()

In [0]:
Testing_df = Testing_df.drop(84, axis=0)
Testing_df = Testing_df.drop(1139, axis=0)
Testing_df = Testing_df.drop(2122, axis=0)

In [0]:
Testing_df[Testing_df.Elapsed_days.astype(int) < 0]

Bining the data in large buckets - 
pd.cut(np.array([1, 7, 5, 4, 6, 3]),
...        3, labels=["bad", "medium", "good"])
[bad, good, medium, medium, good, bad]
Categories (3, object): [bad < medium < good]

In [0]:
bins = [-1, 1, 100, 200, 400, 800, 1600, 3200]
labels = [0, 1, 2, 3, 4, 5, 6]
Testing_df['Binned'] = pd.cut(Testing_df['Elapsed_days'], bins=bins, labels=labels)

Start of the ML algorithm
Predictor Vars are the variables for us to play with down the line.

In [0]:
Testing_df.Binned.describe()


In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

Testing_df.fillna(0, inplace = True)

predictor_vars = ['ApplType', 'SubmissionType', 'SubmissionNo', 'ReviewPriority', 'SubmissionClassCodeID']
data, target = Testing_df[predictor_vars], Testing_df.Binned
Xtrain, Xtest, ytrain, ytest = train_test_split(data, target, random_state=0)

model = RandomForestClassifier(n_estimators=1000)
model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest)
 

In [0]:
from sklearn import metrics
print(metrics.classification_report(ypred, ytest))


#Start Here 



SubmissionClassCodeID	SubmissionClassCode	SubmissionClassCodeDescription

1	BIOEQUIV	Bioequivalence

2	EFFICACY	Efficacy

3	LABELING	Labeling

4	MANUF (CMC)	Manufacturing (CMC)

5	N/A	Not Applicable

6	S	Supplement

7	TYPE 1	Type 1 - New Molecular Entity

8	TYPE 1/4	Type 1 - New Molecular Entity and Type 4 - New Combination

9	TYPE 2	Type 2 - New Active Ingredient

10	TYPE 2/3	Type 2 - New Active Ingredient and Type 3 - New Dosage Form

11	TYPE 2/4	Type 2 New Active Ingredient and Type 4 New Combination

12	TYPE 3	Type 3 - New Dosage Form

13	TYPE 3/4	Type 3 - New Dosage Form and Type 4 - New Combination

14	TYPE 4	Type 4 - New Combination

15	TYPE 5	Type 5 - New Formulation or New Manufacturer

16	TYPE 6	Type 6 - New Indication (no longer used)

17	TYPE 7	Type 7 - Drug Already Marketed without Approved NDA

18	TYPE 8	Type 8 - Partial Rx to OTC Switch

19	UNKNOWN	

20	Unspecified	

21	REMS	REMS

22	TYPE 10	Type 10 - New Indication Submitted as Distinct NDA - Not Consolidated

23	MEDGAS	Medical Gas

24	TYPE 9	Type 9 - New Indication Submitted as Distinct NDA, Consolidated with Original NDA after Approval

25	TYPE 9- BLA	Type 9 - New indication submitted as distinct BLA, consolidated





In [0]:
list(Master_df.columns)

from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier().fit(X, y)